![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)


# CAPSTONE PROJECT: Predicción de precios de las viviendas en Madrid
Limpieza de datos: Pisos.com

Meta de creación de columnas:
- `fecha_descarga`-
- `web`-
- `url`-
- `precio`-
- `barrio`-
- `distrito`-
- `codigo_postal`
- `metros_utiles`
- `metros_construidos`-
- `num_hab`-
- `num_banos`-
- `planta`-
- `tipologia`-
- `obra_nueva`
- `cocina`
- `garage`
- `trastero`
- `orientacion`
- `estado`-
- `antiguedad`
- `consumo`
- `fecha_publicacion`
- `subasta`
- `visitas`
- `amueblado`
- `calefaccion`
- `terraza`
- `balcon`
- `piscina`
- `jardin`
- `aire_acondionado`
- `detalle`

Cuatro datasets recibidos
- data_sales_19102024.xlsx
- data_salesagosto2024.xlsx
- Detalles_datasales19102024.xlsx
- Detalles2_datasales19102024.xlsx

In [629]:
import pandas as pd
import numpy as np
import unicodedata
import re

## DATA FRAMES GENERALES

### TRANSFORMAR COLUMNAS Y MERGE

In [631]:
#Silvia, modifico la ruta para acceder al archivo de datos
df_pisos_10=pd.read_excel('./03-pisos-com/data_sales_19102024.xlsx')
df_pisos_10.head(2)


,title,zone,municipality,price,bedrooms,bathrooms,size,floor,priceByArea,tipology,state,url
0,Apartamento en venta en Paseo de las Acacias,Acacias (Distrito Arganzuela. Madrid Capital),madrid,245.000 €,2 habs.,1 baño,42 m²,3ª planta,5.833 €/m²,NaN,En buen estado,https://www.pisos.com/comprar/apartamento-acacias28005-47581419946_100500/
1,Apartamento en venta en Calle de Carlos y Guillermo Fernández Shaw,Adelfas (Distrito Retiro. Madrid Capital),madrid,767.500 €,3 habs.,2 baños,110 m²,NaN,6.977 €/m²,NaN,NaN,https://www.pisos.com/comprar/apartamento-adelfas28007-45835099732_101800/


In [632]:
df_pisos_10['size'].value_counts(dropna=False)

size
160 m²      136
110 m²       93
65 m²        89
53 m²        76
123 m²       75
           ... 
754 m²        1
775 m²        1
700 m²        1
1.525 m²      1
490 m²        1
Name: count, Length: 431, dtype: int64

In [633]:
df_pisos_08=pd.read_excel('./03-pisos-com/data_salesagosto2024.xlsx')

df_pisos_08.head(2)

,title,zone,municipality,price,bedrooms,bathrooms,size,floor,priceByArea,tipology,state,url
0,Apartamento en venta en Calle de Valdelasierra,Acacias (Distrito Arganzuela. Madrid Capital),madrid,255.000 €,1 hab.,1 baño,71 m²,NaN,3.591 €/m²,NaN,En buen estado,https://www.pisos.com/comprar/apartamento-acacias28005-46704102553_993362/
1,"Apartamento en venta en Paseo de Santa María de la Cabeza, cerca de Paseo de las Delicias",Acacias (Distrito Arganzuela. Madrid Capital),madrid,397.000 €,1 hab.,1 baño,73 m²,3ª planta,5.438 €/m²,NaN,En buen estado,https://www.pisos.com/comprar/apartamento-acacias28045-45078322021_993362/


In [634]:
#df_pisos_08 = pd.read_excel('/Users/AleVC/Library/CloudStorage/GoogleDrive-ale.vaquero.c@gmail.com/My Drive/Nuclio/Máster en Data Science/TFM/PisosCom/data_salesagosto2024.xlsx')
#df_pisos_08.head(2)

In [635]:
df_pisos_10.shape
# 3715, 12

(3715, 12)

In [636]:
df_pisos_10_dupl = df_pisos_10.duplicated().sum()
df_pisos_10_dupl
# 735 filas duplicadas

735

In [637]:
df_pisos_10 = df_pisos_10.drop_duplicates()
df_pisos_10.shape
# 2980, 12

(2980, 12)

In [638]:
df_pisos_08.shape
# 4233, 12

(4233, 12)

In [639]:
df_pisos_08_dupl = df_pisos_08.duplicated().sum()
df_pisos_08_dupl
# 1264 filas duplicadas

1264

In [640]:
df_pisos_08 = df_pisos_08.drop_duplicates()
df_pisos_08.shape
# 2969, 12

(2969, 12)

In [641]:
df_pisos_10['fecha_descarga'] = pd.to_datetime('2024-10-19')
df_pisos_10.head(1)

,title,zone,municipality,price,bedrooms,bathrooms,size,floor,priceByArea,tipology,state,url,fecha_descarga
0,Apartamento en venta en Paseo de las Acacias,Acacias (Distrito Arganzuela. Madrid Capital),madrid,245.000 €,2 habs.,1 baño,42 m²,3ª planta,5.833 €/m²,NaN,En buen estado,https://www.pisos.com/comprar/apartamento-acacias28005-47581419946_100500/,2024-10-19


In [642]:
df_pisos_10.shape
# (2980, 13)

(2980, 13)

In [643]:
df_pisos_08['fecha_descarga'] = pd.to_datetime('2024-08-19')
df_pisos_08.head(1)

,title,zone,municipality,price,bedrooms,bathrooms,size,floor,priceByArea,tipology,state,url,fecha_descarga
0,Apartamento en venta en Calle de Valdelasierra,Acacias (Distrito Arganzuela. Madrid Capital),madrid,255.000 €,1 hab.,1 baño,71 m²,NaN,3.591 €/m²,NaN,En buen estado,https://www.pisos.com/comprar/apartamento-acacias28005-46704102553_993362/,2024-08-19


In [644]:
df_pisos_08.shape
# (2969, 13)

(2969, 13)

In [645]:
df_pisos = df_pisos_10.merge(df_pisos_08, how='outer')
df_pisos.head(2)

,title,zone,municipality,price,bedrooms,bathrooms,size,floor,priceByArea,tipology,state,url,fecha_descarga
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),madrid,250.000 €,1 hab.,1 baño,39 m²,5ª planta,6.410 €/m²,NaN,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19
1,Apartamento en venta en Apóstol Santiago,Apóstol Santiago (Distrito Hortaleza. Madrid Capital),madrid,420.000 €,2 habs.,1 baño,73 m²,NaN,5.753 €/m²,NaN,NaN,https://www.pisos.com/comprar/apartamento-apostol_santiago-41696859245_101800/,2024-10-19


In [646]:
df_pisos.tail(2)

,title,zone,municipality,price,bedrooms,bathrooms,size,floor,priceByArea,tipology,state,url,fecha_descarga
5947,Ático en venta en los Berrocales,Los Berrocales (Distrito Vicálvaro. Madrid Capital),madrid,498.000 €,3 habs.,2 baños,157 m²,NaN,3.171 €/m²,NaN,A estrenar,https://www.pisos.com/comprar/atico-los_berrocales28031-4666816647_109700/,2024-08-19
5948,Ático en venta en los Berrocales,Los Berrocales (Distrito Vicálvaro. Madrid Capital),madrid,498.000 €,3 habs.,2 baños,157 m²,NaN,3.171 €/m²,NaN,A estrenar,https://www.pisos.com/comprar/atico-los_berrocales28031-4666816647_109700/,2024-10-19


In [647]:
df_pisos.shape
# (5949, 13)

(5949, 13)

In [648]:
df_pisos.columns

Index(['title', 'zone', 'municipality', 'price', 'bedrooms', 'bathrooms',
       'size', 'floor', 'priceByArea', 'tipology', 'state', 'url',
       'fecha_descarga'],
      dtype='object')

In [649]:
df_pisos['municipality'].value_counts()

municipality
madrid    5949
Name: count, dtype: int64

In [650]:
# Eliminar la columna 'municipality' ya que solo indica Madrid que es la ubicación de todo el estudio.
df_pisos = df_pisos.drop(['municipality'], axis=1)

In [651]:
# Renombrar columnas
df_pisos = df_pisos.rename(columns={'zone': 'barrio', 'price': 'precio', 'bedrooms': 'num_hab', 'bathrooms': 'num_banos', 'size': 'metros_construidos', 'priceByArea': 'precio_por_area',  'floor': 'planta', 'state': 'estado'})
df_pisos.columns
# ['title', 'barrio', 'precio', 'num_hab', 'num_banos', 'metros_construidos', 'planta', 'precio_por_area', 'tipologia', 'estado', 'url', 'fecha_descarga']

Index(['title', 'barrio', 'precio', 'num_hab', 'num_banos',
       'metros_construidos', 'planta', 'precio_por_area', 'tipology', 'estado',
       'url', 'fecha_descarga'],
      dtype='object')

In [652]:
df_pisos['tipologia'] = df_pisos['title'].str.split().str[0]

In [653]:
df_pisos["tipologia"].value_counts(dropna=False)

tipologia
Piso           4867
Ático           289
Chalet          261
Apartamento     152
Dúplex          142
Casa            114
Estudio          97
Loft             22
Finca             5
Name: count, dtype: int64

In [654]:
df_pisos=df_pisos.drop(columns='tipology')

In [655]:
df_pisos.head(1)

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),250.000 €,1 hab.,1 baño,39 m²,5ª planta,6.410 €/m²,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19,Apartamento


In [656]:
df_pisos['web'] = 'pisos.com'
df_pisos.columns

Index(['title', 'barrio', 'precio', 'num_hab', 'num_banos',
       'metros_construidos', 'planta', 'precio_por_area', 'estado', 'url',
       'fecha_descarga', 'tipologia', 'web'],
      dtype='object')

In [657]:
df_pisos.head(1)

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),250.000 €,1 hab.,1 baño,39 m²,5ª planta,6.410 €/m²,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19,Apartamento,pisos.com


In [658]:
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               5949 non-null   object        
 1   barrio              5949 non-null   object        
 2   precio              5949 non-null   object        
 3   num_hab             5719 non-null   object        
 4   num_banos           5609 non-null   object        
 5   metros_construidos  5930 non-null   object        
 6   planta              3368 non-null   object        
 7   precio_por_area     5925 non-null   object        
 8   estado              3642 non-null   object        
 9   url                 5949 non-null   object        
 10  fecha_descarga      5949 non-null   datetime64[ns]
 11  tipologia           5949 non-null   object        
 12  web                 5949 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 6

In [659]:
hab_count = df_pisos['num_hab'].value_counts(ascending=True)
hab_count

num_hab
14 habs.       1
17 habs.       2
15 habs.       3
12 habs.       5
10 habs.       6
11 habs.       8
9 habs.       14
8 habs.       29
7 habs.       68
6 habs.      128
5 habs.      318
1 hab.       647
4 habs.      848
2 habs.     1677
3 habs.     1965
Name: count, dtype: int64

In [660]:
banos_count = df_pisos['num_banos'].value_counts(ascending=True)
banos_count

num_banos
16 baños       1
13 baños       2
12 baños       3
11 baños       5
10 baños       7
9 baños        7
8 baños       14
7 baños       39
6 baños      103
5 baños      252
4 baños      458
3 baños      918
1 baño      1888
2 baños     1912
Name: count, dtype: int64

In [661]:
planta_count = df_pisos['planta'].value_counts(ascending=True)
planta_count

planta
20ª planta            1
16ª planta            2
14ª planta            3
12ª planta            3
Más de 20 planta      4
15ª planta            4
11ª planta            8
13ª planta            9
10ª planta           12
9ª planta            37
8ª planta            52
7ª planta           114
5ª planta           259
6ª planta           260
4ª planta           439
3ª planta           599
2ª planta           681
1ª planta           881
Name: count, dtype: int64

### DATOS NUMÉRICOS INT: TRANSFORMAR Y RELLENAR NULOS

In [662]:
df_pisos['num_hab'] = df_pisos['num_hab'].str.replace(' habs.', '', regex=True)
df_pisos['num_hab'] = df_pisos['num_hab'].str.replace(' hab.', '', regex=True)
# df_pisos['num_hab'] = df_pisos['num_hab'].fillna(-1)
# df_pisos['num_hab'] = pd.to_numeric(df_pisos['num_hab'], errors='coerce').astype('int64')
df_pisos['num_banos'] = df_pisos['num_banos'].str.replace('baños', '', regex=True)
df_pisos['num_banos'] = df_pisos['num_banos'].str.replace('baño', '', regex=True)
# df_pisos['num_banos'] = df_pisos['num_banos'].fillna(-1)
# df_pisos['num_banos'] = pd.to_numeric(df_pisos['num_banos'], errors='coerce').astype('int64')
df_pisos['planta'] = df_pisos['planta'].str.replace('planta', '', regex=True)
df_pisos['planta'] = df_pisos['planta'].str.replace('ª', '', regex=True)
# df_pisos['planta'] = df_pisos['planta'].fillna(-1)
# df_pisos['planta'] = pd.to_numeric(df_pisos['planta'], errors='coerce').astype('int64')
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               5949 non-null   object        
 1   barrio              5949 non-null   object        
 2   precio              5949 non-null   object        
 3   num_hab             5719 non-null   object        
 4   num_banos           5609 non-null   object        
 5   metros_construidos  5930 non-null   object        
 6   planta              3368 non-null   object        
 7   precio_por_area     5925 non-null   object        
 8   estado              3642 non-null   object        
 9   url                 5949 non-null   object        
 10  fecha_descarga      5949 non-null   datetime64[ns]
 11  tipologia           5949 non-null   object        
 12  web                 5949 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 6

### DATOS CATEGÓRICOS: TRANSFORMAR Y RELLENAR NULOS

In [666]:
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               5949 non-null   object        
 1   barrio              5949 non-null   object        
 2   precio              5949 non-null   object        
 3   num_hab             5719 non-null   object        
 4   num_banos           5609 non-null   object        
 5   metros_construidos  5930 non-null   object        
 6   planta              3368 non-null   object        
 7   precio_por_area     5925 non-null   object        
 8   estado              3642 non-null   object        
 9   url                 5949 non-null   object        
 10  fecha_descarga      5949 non-null   datetime64[ns]
 11  tipologia           5949 non-null   object        
 12  web                 5949 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 6

In [667]:
tipologia_count = df_pisos['tipologia'].value_counts(ascending=True)
tipologia_count

tipologia
Finca             5
Loft             22
Estudio          97
Casa            114
Dúplex          142
Apartamento     152
Chalet          261
Ático           289
Piso           4867
Name: count, dtype: int64

In [668]:
estado_count = df_pisos['estado'].value_counts(ascending=True)
estado_count

estado
A reformar         431
Reformado          533
A estrenar         621
En buen estado    2057
Name: count, dtype: int64

In [669]:
origen_count = df_pisos['web'].value_counts(ascending=True)
origen_count

web
pisos.com    5949
Name: count, dtype: int64

In [670]:
# df_pisos['tipologia'] = df_pisos['tipologia'].fillna('desconocido').astype('category')
# df_pisos['estado'] = df_pisos['estado'].fillna('desconocido').astype('category')
# df_pisos['ref_origen'] = df_pisos['ref_origen'].astype('category')
# df_pisos.info()

### DATOS NUMÉRICOS FLOAT: RELLENAR NULOS Y TRANSFORMAR

In [671]:
precio_decimales = df_pisos[df_pisos['precio'].str.contains(',', na=False)]
precio_decimales
# Se ha comprobado que ningún 'precio' contiene decimales.

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web


In [672]:
metros_decimales = df_pisos[df_pisos['metros_construidos'].str.contains(',', na=False)]
metros_decimales
# Se ha comprobado que ningún 'metros_construidos' contiene decimales.

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web


In [673]:
area_decimales = df_pisos[df_pisos['precio_por_area'].str.contains(',', na=False)]
area_decimales
# Se ha comprobado que ningún 'precio_por_area' contiene decimales.

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web


In [674]:
#compruebo los valores de metros construidos
df_pisos['metros_construidos'].describe()

count      5930
unique      474
top       70 m²
freq        100
Name: metros_construidos, dtype: object

In [675]:
df_pisos['metros_construidos'].value_counts(dropna=False)

metros_construidos
70 m²     100
90 m²      80
80 m²      79
60 m²      77
110 m²     75
         ... 
577 m²      1
781 m²      1
428 m²      1
699 m²      1
561 m²      1
Name: count, Length: 475, dtype: int64

In [676]:
df_pisos.query("url=='https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/'")

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),250.000 €,1,1,39 m²,5,6.410 €/m²,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19,Apartamento,pisos.com


In [677]:
df_pisos['precio'] = df_pisos['precio'].str.replace(r'\D', '', regex=True)
df_pisos['metros_construidos'] = df_pisos['metros_construidos'].str.replace(r'\D', '', regex=True)

df_pisos['precio_por_area'] = df_pisos['precio_por_area'].str.replace(r'\D', '', regex=True)
df_pisos.head(1)

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),250000,1,1,39,5,6410,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19,Apartamento,pisos.com


In [678]:
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               5949 non-null   object        
 1   barrio              5949 non-null   object        
 2   precio              5949 non-null   object        
 3   num_hab             5719 non-null   object        
 4   num_banos           5609 non-null   object        
 5   metros_construidos  5930 non-null   object        
 6   planta              3368 non-null   object        
 7   precio_por_area     5925 non-null   object        
 8   estado              3642 non-null   object        
 9   url                 5949 non-null   object        
 10  fecha_descarga      5949 non-null   datetime64[ns]
 11  tipologia           5949 non-null   object        
 12  web                 5949 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 6

In [679]:
# df_pisos['precio'] = pd.to_numeric(df_pisos['precio'], errors='coerce')
# df_pisos['precio'] = df_pisos['precio'].astype('float64').fillna(-1)
# df_pisos['metros_construidos'] = pd.to_numeric(df_pisos['metros_construidos'], errors='coerce')
# df_pisos['metros_construidos'] = df_pisos['metros_construidos'].astype('float64').fillna(-1)
# df_pisos['precio_por_area'] = pd.to_numeric(df_pisos['precio_por_area'], errors='coerce')
# df_pisos['precio_por_area'] = df_pisos['precio_por_area'].astype('float64').fillna(-1)
# df_pisos.info()

### BARRIOS Y DISTRITOS

In [680]:
df_pisos.head(2)

,title,barrio,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),250000,1,1,39,5,6410,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19,Apartamento,pisos.com
1,Apartamento en venta en Apóstol Santiago,Apóstol Santiago (Distrito Hortaleza. Madrid Capital),420000,2,1,73,NaN,5753,NaN,https://www.pisos.com/comprar/apartamento-apostol_santiago-41696859245_101800/,2024-10-19,Apartamento,pisos.com


In [681]:
df_pisos = df_pisos.rename(columns={'barrio': 'ubicacion'})
df_pisos.head(2)

,title,ubicacion,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),250000,1,1,39,5,6410,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19,Apartamento,pisos.com
1,Apartamento en venta en Apóstol Santiago,Apóstol Santiago (Distrito Hortaleza. Madrid Capital),420000,2,1,73,NaN,5753,NaN,https://www.pisos.com/comprar/apartamento-apostol_santiago-41696859245_101800/,2024-10-19,Apartamento,pisos.com


In [682]:
df_pisos = df_pisos.reset_index(drop=True)

In [683]:
#df_barrios = pd.read_csv('./distritos-y-barrios-limpio.csv')


In [684]:
df_barrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [685]:
#df_barrios = pd.read_csv('/Users/AleVC/Library/CloudStorage/GoogleDrive-ale.vaquero.c@gmail.com/My Drive/Nuclio/Máster en Data Science/TFM/Datos-adicionales/Datos-limpios/distritos-y-barrios-limpio.csv')
df_barrios.head(2)

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal
0,1,Centro,11,1,Palacio,28013
1,1,Centro,12,2,Embajadores,28012


In [686]:
distritos_unicos = df_barrios['distrito'].unique()
distritos_unicos

array(['Centro', 'Arganzuela', 'Retiro', 'Salamanca', 'Chamartín',
       'Tetuán', 'Chamberí', 'Fuencarral-El Pardo', 'Moncloa-Aravaca',
       'Latina', 'Carabanchel', 'Usera', 'Puente de Vallecas',
       'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
       ' Villa de Vallecas', ' Vicálvaro', ' San Blas-Canillejas',
       ' Barajas'], dtype=object)

In [687]:
barrios_unicos = df_barrios['barrio'].unique()
barrios_unicos

array(['Palacio', 'Embajadores', 'Cortes', 'Justicia', 'Universidad',
       'Sol', 'Imperial', 'Acacias', 'Chopera', 'Legazpi', 'Delicias',
       'Palos de Moguer', 'Atocha', 'Pacífico', 'Adelfas', 'Estrella',
       'Ibiza', 'Jerónimos', 'Niño Jesus', 'Recoletos', 'Goya',
       'Fuente del Berro', 'Guindalera', 'Lista', 'Castellana', 'El Viso',
       'Prosperidad', 'Ciudad Jardín', 'Hispanoamérica', 'Nueva España',
       'Castilla', 'Bellas Vistas', 'Cuatro Caminos', 'Castillejos',
       'Almenara', 'Valdeacederas', 'Berruguete', 'Gaztambide',
       'Arapiles', 'Trafalgar', 'Almagro', 'Ríos Rosas', 'Vallehermoso',
       'El Pardo', 'Fuentelarreina', 'Peñagrande', 'Pilar', 'La Paz',
       'Valverde', 'Mirasierra', 'El Goloso', 'Casa de Campo',
       'Argüelles', 'Ciudad Universitaria', 'Valdezarza', 'Valdemarín',
       'El Plantío', 'Aravaca', 'Los Cármenes', 'Puerta del Ángel',
       'Lucero', 'Aluche', 'Campamento', 'Cuatro Vientos', 'Las Águilas',
       'Comillas', 'Opa

In [688]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")
    return s  # Convertir a minúsculas
       

In [689]:
nuevas_columnas= ["barrio","cod_barrio","distrito","cod_distrito","codigo_Postal"]
df_pisos[nuevas_columnas]=np.nan
zona_barrio_dict = {
    'Sanchinarro': {     
        'barrio': 'Valdefuentes'
    },
    'San Andrés': {       
        'barrio': 'Villaverde Alto'
    },
    'Virgen del Cortijo': {      
        'barrio': 'Valdefuentes'
    },
    'Montecarmelo': {       
        'barrio': 'Mirasierra'
    }
    ,
    'Villaverde Bajo': {       
        'barrio': 'Los Rosales'
    },
    'villaverdebajo': {       
        'barrio': 'Los Rosales'
    }
}


# Cambiar el tipo de dato de las columnas "barrio" y "distrito" a object
df_pisos["barrio"] = df_pisos["barrio"].astype(object)
df_pisos["distrito"] = df_pisos["distrito"].astype(object)
df_pisos["cod_barrio"] = df_pisos["cod_barrio"].astype(object)
df_pisos["cod_distrito"] = df_pisos["cod_distrito"].astype(object)
df_pisos["codigo_Postal"] = df_pisos["codigo_Postal"].astype(object)
df_barrios.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in df_barrios.loc[:, 'distrito']]
df_barrios.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in df_barrios.loc[:, 'barrio']]

def clasificar_distrito_y_barrio_por_columnas(df, df_barrios):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Recorrer las columnas en orden de prioridad: titulo, nombre_oficina
        for columna in ['ubicacion']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)

                 # Reemplazar valores específicos en la ubicación normalizada
                if 'sanandres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanandres', 'villaverdealto')
                elif 'salvador' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('salvador', 'elsalvador')
                elif 'san andres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('san andres', 'villaverdealto')
                elif 'palos de moguer' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('palos de moguer', 'palosdemoguer')
                elif 'virgendelcortijo-manoteras' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('virgendelcortijo-manoteras', 'valdefuentes')
                elif 'sanchinarro' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanchinarro', 'valdefuentes')
                elif 'montecarmelo' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('montecarmelo', 'mirasierra')
                elif 'ambroz' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('ambroz', 'cascohistoricodevicalvaro')
                elif 'carmenes' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('carmenes', 'loscarmenes')
                elif 'paudecarabanchel' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('paudecarabanchel', 'buenavista')
                elif 'centrohistorico' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('centrohistorico', 'cascohistoricodevicalvaro')
                
                # Verificamos si el valor normalizado contiene el nombre de un barrio
                barrio_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal
                # Verificamos si el valor normalizado contiene el nombre de un distrito
                distrito_encontrado = next((distrito for distrito in df_barrios["distrito_n"] if distrito in valor_normalizado), None)
                if distrito_encontrado and pd.isna(fila["distrito"]):
                    df.at[idx, "distrito"] = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")["distrito"].values[0]
                    result = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")                    
                    cod_distrito = result["cod_distrito"].values[0]
                    df.at[idx, "cod_distrito"] = cod_distrito
            else:
                #zona_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                barrio_encontrado = next((zona_barrio_dict[zona]["barrio"] for zona in zona_barrio_dict if zona_barrio_dict[zona]["barrio"] in df_barrios["barrio_n"]), None)
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal

In [690]:
df_barrios.query("distrito=='Hortaleza'")

,cod_distrito,distrito,cod_barrio,cod_barrio_2,barrio,codigo_Postal,distrito_n,barrio_n
100,16,Hortaleza,161,1,Palomas,28043,hortaleza,palomas
101,16,Hortaleza,162,2,Valdefuentes,28055,hortaleza,valdefuentes
102,16,Hortaleza,163,3,Canillas,28043,hortaleza,canillas
103,16,Hortaleza,164,4,Pinar del Rey,28033,hortaleza,pinardelrey
104,16,Hortaleza,165,5,Apostol Santiago,28033,hortaleza,apostolsantiago
105,16,Hortaleza,166,6,Piovera,28033,hortaleza,piovera
106,16,Hortaleza,167,7,Valdebebas-Valdefuentes,28055,hortaleza,valdebebas-valdefuentes


In [691]:
clasificar_distrito_y_barrio_por_columnas(df_pisos, df_barrios)

In [692]:
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               5949 non-null   object        
 1   ubicacion           5949 non-null   object        
 2   precio              5949 non-null   object        
 3   num_hab             5719 non-null   object        
 4   num_banos           5609 non-null   object        
 5   metros_construidos  5930 non-null   object        
 6   planta              3368 non-null   object        
 7   precio_por_area     5925 non-null   object        
 8   estado              3642 non-null   object        
 9   url                 5949 non-null   object        
 10  fecha_descarga      5949 non-null   datetime64[ns]
 11  tipologia           5949 non-null   object        
 12  web                 5949 non-null   object        
 13  barrio              5922 non-null   object      

In [693]:
df_pisos.query("cod_barrio.isnull()")

,title,ubicacion,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal
994,"Piso en venta en Avenida Levantge, S/N",Los Berrocales (Distrito Vicálvaro. Madrid Capital),354000,3,2,110,1,3218,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28052-4592114246_109700/,2024-10-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
1188,"Piso en venta en Calle Berrocales Ix, 461",Los Berrocales (Distrito Vicálvaro. Madrid Capital),297400,2,2,77,NaN,3862,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4596004879_109700/,2024-08-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
1189,"Piso en venta en Calle Berrocales Ix, 461",Los Berrocales (Distrito Vicálvaro. Madrid Capital),301500,2,2,77,NaN,3915,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4849665223_109700/,2024-10-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
1190,"Piso en venta en Calle Berrocales Ix, 461",Los Berrocales (Distrito Vicálvaro. Madrid Capital),369900,3,2,99,NaN,3736,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4510709962_109700/,2024-08-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
1191,"Piso en venta en Calle Berrocales Ix, 461",Los Berrocales (Distrito Vicálvaro. Madrid Capital),371900,3,2,99,NaN,3756,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4891530918_109700/,2024-10-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
1360,"Piso en venta en Calle Parcela Rc.4.6.2. Sector Uzpp. 02.04, S/N",Los Berrocales (Distrito Vicálvaro. Madrid Capital),320500,2,2,93,1,3446,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-3500496548_109700/,2024-08-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
1361,"Piso en venta en Calle Parcela Rc.4.6.2. Sector Uzpp. 02.04, S/N",Los Berrocales (Distrito Vicálvaro. Madrid Capital),358500,3,2,107,1,3350,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4568166771_109700/,2024-08-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
5619,Piso en venta en los Berrocales,Los Berrocales (Distrito Vicálvaro. Madrid Capital),279000,2,2,90,1,3100,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4677433362_109700/,2024-08-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
5620,Piso en venta en los Berrocales,Los Berrocales (Distrito Vicálvaro. Madrid Capital),279000,2,2,90,1,3100,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4677433362_109700/,2024-10-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN
5621,Piso en venta en los Berrocales,Los Berrocales (Distrito Vicálvaro. Madrid Capital),287000,2,2,92,1,3119,A estrenar,https://www.pisos.com/comprar/piso-los_berrocales28031-4588865732_109700/,2024-08-19,Piso,pisos.com,NaN,NaN,Vicálvaro,19,NaN


# Función para identificar el distrito
def extraer_distrito(ubicacion, distritos_unicos):
    for distrito in distritos_unicos:
        if distrito in ubicacion:
            return distrito
    return np.nan  # Si no se encuentra un distrito

# Aplicar la función a la columna 'ubicacion' y crear la nueva columna 'distrito'
df_pisos['distrito'] = df_pisos['ubicacion'].apply(lambda x: extraer_distrito(x, distritos_unicos))
df_pisos.head(2)


# Función para identificar el barrio
def extraer_barrio(ubicacion, barrios_unicos):
    for barrio in barrios_unicos:
        if barrio in ubicacion:
            return barrio
    return np.nan  # Si no se encuentra un barrio

# Aplicar la función a la columna 'ubicacion' y crear la nueva columna 'barrio'
df_pisos['barrio'] = df_pisos['ubicacion'].apply(lambda x: extraer_barrio(x, barrios_unicos))
df_pisos.head(2)

In [694]:
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               5949 non-null   object        
 1   ubicacion           5949 non-null   object        
 2   precio              5949 non-null   object        
 3   num_hab             5719 non-null   object        
 4   num_banos           5609 non-null   object        
 5   metros_construidos  5930 non-null   object        
 6   planta              3368 non-null   object        
 7   precio_por_area     5925 non-null   object        
 8   estado              3642 non-null   object        
 9   url                 5949 non-null   object        
 10  fecha_descarga      5949 non-null   datetime64[ns]
 11  tipologia           5949 non-null   object        
 12  web                 5949 non-null   object        
 13  barrio              5922 non-null   object      

In [695]:
df_pisos["distrito"] = df_pisos["distrito"].str.strip()


In [696]:
distrito_count = df_pisos['distrito'].value_counts(ascending=True)
distrito_count

distrito
Moratalaz                46
Usera                    76
Barajas                  83
Villa de Vallecas        98
Vicálvaro                99
Villaverde              108
Arganzuela              133
Hortaleza               157
Latina                  177
San Blas-Canillejas     191
Moncloa-Aravaca         220
Ciudad Lineal           230
Puente de Vallecas      241
Fuencarral-El Pardo     257
Retiro                  267
Carabanchel             282
Tetuán                  333
Chamberí                380
Chamartín               458
Salamanca               992
Centro                 1121
Name: count, dtype: int64

In [697]:
barrio_count = df_pisos['barrio'].value_counts(ascending=True)
barrio_count

barrio
Valdebernardo      2
Valderrivas        3
Orcasur            3
Pavones            4
Vinateros          4
                ... 
Palacio          210
Castellana       247
El Pardo         257
Goya             293
Universidad      302
Name: count, Length: 123, dtype: int64

In [698]:
# df_pisos['distrito'] = df_pisos['distrito'].fillna('desconocido').astype('category')
# df_pisos['barrio'] = df_pisos['barrio'].fillna('desconocido').astype('category')
# df_pisos.info()

In [699]:
# Ajustar pandas para mostrar contenido completo
pd.set_option('display.max_colwidth', None)
df_pisos.head()

,title,ubicacion,precio,num_hab,num_banos,metros_construidos,planta,precio_por_area,estado,url,fecha_descarga,tipologia,web,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en A Tan Solo Metros del Teatro de La Latina,Palacio (Distrito Centro. Madrid Capital),250000,1,1,39,5,6410,NaN,https://www.pisos.com/comprar/apartamento-centro_palacio28005-47597559176_109100/,2024-10-19,Apartamento,pisos.com,Palacio,11,Centro,1,28013
1,Apartamento en venta en Apóstol Santiago,Apóstol Santiago (Distrito Hortaleza. Madrid Capital),420000,2,1,73,NaN,5753,NaN,https://www.pisos.com/comprar/apartamento-apostol_santiago-41696859245_101800/,2024-10-19,Apartamento,pisos.com,Apostol Santiago,165,Hortaleza,16,28033
2,Apartamento en venta en Argüelles,Argüelles (Distrito Moncloa-Aravaca. Madrid Capital),3990000,8,4,372,NaN,10725,NaN,https://www.pisos.com/comprar/apartamento-moncloa_aravaca_arguelles28008-48334738432_101800/,2024-10-19,Apartamento,pisos.com,Argüelles,92,Moncloa-Aravaca,9,28008
3,Apartamento en venta en Argüelles,Argüelles (Distrito Moncloa-Aravaca. Madrid Capital),515000,2,1,75,NaN,6866,NaN,https://www.pisos.com/comprar/apartamento-moncloa_aravaca_arguelles28008-48429886093_101800/,2024-10-19,Apartamento,pisos.com,Argüelles,92,Moncloa-Aravaca,9,28008
4,Apartamento en venta en Atalaya,Atalaya (Distrito Ciudad Lineal. Madrid Capital),1220000,5,3,331,NaN,3685,NaN,https://www.pisos.com/comprar/apartamento-ciudad_lineal_atalaya-43413667133_101800/,2024-10-19,Apartamento,pisos.com,Atalaya,158,Ciudad Lineal,15,28027


In [700]:
df_pisos.shape
# (5949, 15)

(5949, 18)

In [701]:
#df_barrios = pd.read_csv('/Users/AleVC/Library/CloudStorage/GoogleDrive-ale.vaquero.c@gmail.com/My Drive/Nuclio/Máster en Data Science/TFM/Datos-adicionales/Datos-limpios/distritos-y-barrios-limpio.csv')
#df_barrios.head()

df_pisos = pd.merge(df_pisos, df_barrios, on=['barrio', 'distrito'], how='left')
df_pisos.shape
# (5949, 17)

df_pisos.head(1)

## COLUMNAS DE DF PISOS

### ASIGNAR COLUMNAS NUEVAS PARA EL FUTURO MERGE CON DF DETALLES

In [702]:
df_pisos.rename(columns={'num_hab': 'habitaciones', 'num_banos': 'banos', 'metros_construidos': 'm2_constr', 'precio_por_area': 'EUR/m2'}, inplace=True)
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           5949 non-null   object        
 1   ubicacion       5949 non-null   object        
 2   precio          5949 non-null   object        
 3   habitaciones    5719 non-null   object        
 4   banos           5609 non-null   object        
 5   m2_constr       5930 non-null   object        
 6   planta          3368 non-null   object        
 7   EUR/m2          5925 non-null   object        
 8   estado          3642 non-null   object        
 9   url             5949 non-null   object        
 10  fecha_descarga  5949 non-null   datetime64[ns]
 11  tipologia       5949 non-null   object        
 12  web             5949 non-null   object        
 13  barrio          5922 non-null   object        
 14  cod_barrio      5922 non-null   object        
 15  dist

In [703]:
 df_pisos['url'].duplicated().sum()

# 1213

1213

In [704]:
df_pisos = df_pisos.drop_duplicates(subset=['url'])
df_pisos.shape
# (4736, 17)

(4736, 18)

In [705]:
df_pisos.columns

Index(['title', 'ubicacion', 'precio', 'habitaciones', 'banos', 'm2_constr',
       'planta', 'EUR/m2', 'estado', 'url', 'fecha_descarga', 'tipologia',
       'web', 'barrio', 'cod_barrio', 'distrito', 'cod_distrito',
       'codigo_Postal'],
      dtype='object')

In [706]:
df_pisos['m2_constr'].value_counts(dropna=False)

m2_constr
70     82
60     67
80     67
90     65
120    61
       ..
352     1
575     1
750     1
365     1
518     1
Name: count, Length: 474, dtype: int64

In [707]:
df_pisos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4736 entries, 0 to 5947
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           4736 non-null   object        
 1   ubicacion       4736 non-null   object        
 2   precio          4736 non-null   object        
 3   habitaciones    4546 non-null   object        
 4   banos           4471 non-null   object        
 5   m2_constr       4723 non-null   object        
 6   planta          2739 non-null   object        
 7   EUR/m2          4719 non-null   object        
 8   estado          2934 non-null   object        
 9   url             4736 non-null   object        
 10  fecha_descarga  4736 non-null   datetime64[ns]
 11  tipologia       4736 non-null   object        
 12  web             4736 non-null   object        
 13  barrio          4717 non-null   object        
 14  cod_barrio      4717 non-null   object        
 15  distrito 

In [708]:
df_pisos["precio"].describe()

count       4736
unique      1091
top       829000
freq          38
Name: precio, dtype: object

In [709]:
df_pisos["precio"].value_counts(dropna=False)

precio
829000     38
859000     35
650000     34
950000     34
1200000    31
           ..
412000      1
1475000     1
336000      1
638000      1
498000      1
Name: count, Length: 1091, dtype: int64

In [710]:
df_pisos.query("precio==''")

,title,ubicacion,precio,habitaciones,banos,m2_constr,planta,EUR/m2,estado,url,fecha_descarga,tipologia,web,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal
683,"Estudio en venta en Calle de Camarillas, 4",Casco Histórico de Barajas (Distrito Barajas. Madrid Capital),,1,1,50,NaN,NaN,En buen estado,https://www.pisos.com/comprar/estudio-casco_historico_de_barajas-46742905658_521139/,2024-08-19,Estudio,pisos.com,Casco Histórico de Barajas,213,Barajas,21,28042
767,Finca rústica en venta en Amposta,Pueblo Nuevo (Distrito Ciudad Lineal. Madrid Capital),,NaN,NaN,NaN,NaN,NaN,NaN,https://www.pisos.com/comprar/finca_rustica-ciudad_lineal_pueblo_nuevo28017-39190283168_100500/,2024-10-19,Finca,pisos.com,Pueblo Nuevo,152,Ciudad Lineal,15,28017
1248,"Piso en venta en Calle Doctor Esquerdo, cerca de Paseo del Marqués de Zafra",Adelfas (Distrito Retiro. Madrid Capital),,1,1,55,3,NaN,En buen estado,https://www.pisos.com/comprar/piso-adelfas28007-40845633641_526615/,2024-08-19,Piso,pisos.com,Adelfas,32,Retiro,3,28007
3535,"Piso en venta en Calle del Real Betis, 11",Puerta Bonita (Distrito Carabanchel. Madrid Capital),,3,2,104,1,NaN,A estrenar,https://www.pisos.com/comprar/piso-puerta_bonita28025-4281208855_109700/,2024-08-19,Piso,pisos.com,Puerta Bonita,115,Carabanchel,11,28025
5731,"Ático en venta en Calle de Aquitania, s/n",Arcos (Distrito San Blas. Madrid Capital),,1,1,69,NaN,NaN,A estrenar,https://www.pisos.com/comprar/atico-arcos28032-4535639357_109700/,2024-08-19,Ático,pisos.com,Arcos,204,San Blas-Canillejas,20,28037


In [711]:
#elimino los pisoso que no tienen precio

In [712]:
df_pisos.shape

(4736, 18)

In [713]:
# Eliminar las filas donde 'precio' es una cadena vacía
df_pisos = df_pisos[df_pisos['precio'] != '']

In [714]:
df_pisos.shape

(4731, 18)

In [715]:
df_pisos["precio"]=df_pisos["precio"].astype(float)

In [716]:
#convertir a float

In [729]:
df_pisos.loc[df_pisos['title'].str.contains('Chalet en venta en Calle de Pléyades, s/n'), ['calle','m2_constr','habitaciones','banos']] = ['Calle de Pléyades',513,5,6]

df_pisos.loc[df_pisos['url'].str.contains('https://www.pisos.com/comprar/piso-latina_aluche28047-4568954085_109700/'), ['calle','m2_constr','planta']] = ['Calle de Camarena',153,0]
df_pisos.loc[df_pisos['url'].str.contains('https://www.pisos.com/comprar/piso-latina_aluche28047-4505819831_109700/'), ['calle','m2_constr','planta']] = ['Calle de Camarena',107,1]
df_pisos.loc[df_pisos['url'].str.contains('https://www.pisos.com/comprar/piso-latina_aluche28047-4696641924_109700/'), ['calle','m2_constr','planta']] = ['Calle de Camarena',120,2]
df_pisos.loc[df_pisos['url'].str.contains('https://www.pisos.com/comprar/piso-latina_aluche28047-4715872594_109700/'), ['calle','m2_constr','planta']] = ['Calle de Camarena',120,2]
df_pisos.loc[df_pisos['url'].str.contains('https://www.pisos.com/comprar/atico-latina_aluche28047-4826295847_109700/'), ['calle','m2_constr']] = ['Calle de Camarena',123]

In [730]:
df_pisos.query("m2_constr.isnull()")

,title,ubicacion,precio,habitaciones,banos,m2_constr,planta,EUR/m2,estado,url,fecha_descarga,tipologia,web,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,calle
2622,Piso en venta en Calle de Ribadavia,Pilar (Distrito Fuencarral-El Pardo. Madrid Capital),357500.0,4,2,NaN,1,NaN,NaN,https://www.pisos.com/comprar/piso-pilar28029-41728621605_106000/,2024-08-19,Piso,pisos.com,El Pardo,81,Fuencarral-El Pardo,8,28048,NaN
4499,Piso en venta en Madrid,Fontarrón (Distrito Moratalaz. Madrid Capital),599000.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/,2024-10-19,Piso,pisos.com,Fontarrón,145,Moratalaz,14,28030,NaN


In [731]:
# Guardar el DataFrame como un archivo CSV con delimitador punto y coma
df_pisos.to_csv('pisoscom_limpio.csv', sep=';', index=False)